In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np

In [2]:
from config import Cen_API

In [3]:
import utilcalcs as calc
import geo_agg as geo
from county_codes import PEP_DATE_CODE,stco

## User-defined parameters

In [4]:
#Variables and predicates for both start/end years
acs5 = 'acs5'
acs1 = 'acs1'

col_1_d = f'group(DP04)'
col_5_b = f'group(B23001)'
col_5_d = f'GEO_ID,DP02_0092E,DP02_0092M,DP03_0002E,DP03_0002M,DP05_0001E,DP05_0001M'

year1 = '2018'
year0 = '2010'

### Data download functions for PEP and ACS

In [5]:
def get_pep(year1,year0):
    cols = f'GEO_ID,DATE_CODE,POP'
    url = f'https://api.census.gov/data/{year1}/pep/population?get={cols}&for=county:*&in=state:*&key={Cen_API}'
    resp = requests.request('GET', url).content
    df = pd.DataFrame(json.loads(resp)[1:])
    df.columns = json.loads(resp)[0]
    
    df['POP'] = df['POP'].astype(float)
    dff = df.pivot(index='GEO_ID',columns='DATE_CODE',values='POP')
    dff = dff.reset_index(level='GEO_ID',col_level=0)
    dff = dff[['GEO_ID',PEP_DATE_CODE[year0],PEP_DATE_CODE[year1]]]
    dff = dff.rename(columns={PEP_DATE_CODE[year0]:'PopTot_Y0E',PEP_DATE_CODE[year1]:'PopTot_Y1E'})
    
    dff = dff[dff['GEO_ID'].isin(stco)]
    dff['PopTot_Y0Y1E'] = dff.PopTot_Y1E - dff.PopTot_Y0E
    dff['PopP_Y0Y1E'] = dff.PopTot_Y0Y1E / dff.PopTot_Y0E
    
    return dff

def get_acs(year,acs_source,**cols):
    for x in cols:
        if x == 'b':
            url_b = f"https://api.census.gov/data/{year}/acs/{acs_source}?get={cols['b']}&for=county:*&in=state:*&key={Cen_API}"
            resp_b = requests.request('GET', url_b).content
            df_b = pd.DataFrame(json.loads(resp_b)[1:])
            df_b.columns = json.loads(resp_b)[0]
        elif x == 'd':
            url_d = f"https://api.census.gov/data/{year}/acs/{acs_source}/profile?get={cols['d']}&for=county:*&in=state:*&key={Cen_API}"
            resp_d = requests.request('GET', url_d).content
            df_d = pd.DataFrame(json.loads(resp_d)[1:])
            df_d.columns = json.loads(resp_d)[0]
        else:
            pass
    
    if 'b' in cols and 'd' in cols:
        df = pd.merge(df_b,df_d,how='left',on='GEO_ID')
        df = df[df['GEO_ID'].isin(stco)]
        return df
    elif 'b' in cols and 'd' not in cols:
        df_b = df_b[df_b['GEO_ID'].isin(stco)]
        return df_b
    else:
        df_d = df_d[df_d['GEO_ID'].isin(stco)]
        return df_d
        
    
def clean_data(df,var):
    dff = df[var].copy()
    var_num = var[1:]
    for col in var_num:
        dff[col] = dff[col].astype(float)
    dff = dff.replace([999999999, 555555555, 333333333, 222222222,\
                    666666666, 888888888, -999999999, -555555555,\
                    -333333333, -222222222, -666666666, -888888888], np.nan)
    return dff

## Population Estimates Program - estimates

In [6]:
df_pep = get_pep(year1,year0)

In [7]:
df_pep.head()

DATE_CODE,GEO_ID,PopTot_Y0E,PopTot_Y1E,PopTot_Y0Y1E,PopP_Y0Y1E
308,0500000US09001,916829.0,943823.0,26994.0,0.029443
310,0500000US09005,189927.0,181111.0,-8816.0,-0.046418
312,0500000US09009,862477.0,857620.0,-4857.0,-0.005631
1775,0500000US34003,905116.0,936692.0,31576.0,0.034886
1780,0500000US34013,783969.0,799767.0,15798.0,0.020151


In [8]:
df_pep.shape

(31, 5)

In [9]:
#pep.to_csv("pep_test.csv") - for QA

## ACS Data Download

### Variables

In [35]:
#FTotal Population - for calculation of foreign-born share
PopTot = ['DP05_0001E','DP05_0001M']

#Foreign-Born Population - for calculation
PopFB = ['DP02_0092E','DP02_0092M']

#Tot Labor Force - to rename
LFTot = ['DP03_0002E','DP03_0002M']

#Age 25 to 54 in Labor Force - to caluclate
LF2554E = ['B23001_025E','B23001_032E','B23001_111E','B23001_118E','B23001_039E','B23001_125E','B23001_046E','B23001_132E']
LF2554M = ['B23001_025M','B23001_032M','B23001_111M','B23001_118M','B23001_039M','B23001_125M','B23001_046M','B23001_132M']

#Age 65+ in Labor Force - to calculate
LFO65E = ['B23001_074E','B23001_079E','B23001_084E','B23001_160E','B23001_165E','B23001_170E']
LFO65M = ['B23001_074M','B23001_079M','B23001_084M','B23001_160M','B23001_165M','B23001_170M']

#Total Housing Units - to rename
HouTot = ['DP04_0001E','DP04_0001M']

#Total Housing Units by Owner vs. Renter - to rename
HouO = ['DP04_0046E','DP04_0046M']
HouR = ['DP04_0047E','DP04_0047M']
HouV = ['DP04_0003E','DP04_0003M']

#Total Housing Units by Building Size - to calculate
Hou1UE = ['DP04_0007E','DP04_0008E']
Hou1UM = ['DP04_0007M','DP04_0008M']
Hou24UE = ['DP04_0009E','DP04_0010E']
Hou24UM = ['DP04_0009M','DP04_0010M']
Hou5UE = ['DP04_0011E','DP04_0012E','DP04_0013E']
Hou5UM = ['DP04_0011M','DP04_0012M','DP04_0013M']

HouU = Hou1UE + Hou1UM + Hou24UE + Hou24UM + Hou5UE + Hou5UM

#List of all variables used for calculation + total labor force variables - replace the total pop 16+ variables
var_data_Y1 = ['GEO_ID'] + PopTot + PopFB + LFTot + LF2554E + LF2554M + LFO65E + LFO65M + HouTot + HouO + HouR + HouV + HouU 
var_data_Y0 = ['GEO_ID'] + LFTot + LF2554E + LF2554M + LFO65E + LFO65M 

#Variables to drop for subregion and region
var_agg_todrop = ['PopTot_Y0M','PopTot_Y0C','PopTot_Y1M','PopTot_Y1C','PopTot_Y0Y1C','PopTot_Y0Y1M','PopP_Y0Y1C','PopP_Y0Y1M']

## ACS Current [End] Year (Year 1)

In [11]:
co_5 = get_acs(year1,acs5,b=col_5_b,d=col_5_d,)
co_5.head()

,GEO_ID,B23001_001E,B23001_001M,B23001_002E,B23001_002M,B23001_003E,B23001_003M,B23001_004E,B23001_004M,B23001_005E,...,state_x,county_x,DP02_0092E,DP02_0092M,DP03_0002E,DP03_0002M,DP05_0001E,DP05_0001M,state_y,county_y
277,0500000US34025,506127,480,243309,393,16803,382,5996,462,18,...,34,025,83067,2180,333518,2003,623387,-555555555,34,025
278,0500000US34037,117248,240,57857,190,3884,202,1205,188,0,...,34,037,11236,686,80059,814,142298,-555555555,34,037
280,0500000US34013,624970,538,295847,428,20470,441,4429,432,5,...,34,013,209992,3292,411611,2517,793555,-555555555,34,013
281,0500000US34029,466075,433,220607,311,13416,284,4808,376,39,...,34,029,46813,1622,272945,1912,591939,-555555555,34,029
284,0500000US34031,396321,432,190296,332,13621,333,3419,365,0,...,34,031,148834,3324,251608,1859,504041,-555555555,34,031


In [12]:
co_1 = get_acs(year1,acs1,d=col_1_d)
co_1.head()

,DP04_0039M,DP04_0039PE,DP04_0039PM,DP04_0040E,DP04_0040M,DP04_0040PE,DP04_0040PM,DP04_0041E,DP04_0041M,DP04_0041PE,...,DP04_0037MA,DP04_0037PEA,DP04_0037PMA,DP04_0038EA,DP04_0038MA,DP04_0038PEA,DP04_0038PMA,DP04_0039EA,state,county
7,1709,2.7,0.5,48841,3340,13.0,0.9,87944,3798,23.5,...,None,(X),(X),None,None,None,(X),None,09,001
59,2110,6.6,0.7,80397,3667,28.3,1.3,101531,4435,35.7,...,None,(X),(X),None,None,None,(X),None,34,017
63,601,2.1,0.7,13640,1926,16.0,2.2,21460,2090,25.1,...,None,(X),(X),None,None,None,(X),None,36,111
120,1570,2.7,0.4,52424,3650,14.3,1.0,110377,4479,30.0,...,None,(X),(X),None,None,None,(X),None,09,009
169,2245,5.7,0.7,63247,4213,19.8,1.3,88168,4446,27.7,...,None,(X),(X),None,None,None,(X),None,34,013


In [13]:
dfY1 = pd.merge(co_5,co_1,how='left',on='GEO_ID')
dfY1 = clean_data(dfY1,var_data_Y1)

In [14]:
dfY1.head()

,GEO_ID,DP05_0001E,DP05_0001M,DP02_0092E,DP02_0092M,DP03_0002E,DP03_0002M,B23001_025E,B23001_032E,B23001_111E,...,DP04_0009E,DP04_0010E,DP04_0009M,DP04_0010M,DP04_0011E,DP04_0012E,DP04_0013E,DP04_0011M,DP04_0012M,DP04_0013M
0,0500000US34025,623387.0,NaN,83067.0,2180.0,333518.0,2003.0,15541.0,15335.0,13545.0,...,10058.0,7461.0,1854.0,1463.0,10599.0,10959.0,23309.0,2126.0,1577.0,2287.0
1,0500000US34037,142298.0,NaN,11236.0,686.0,80059.0,814.0,3542.0,3435.0,2793.0,...,1153.0,1272.0,539.0,513.0,1905.0,1461.0,1106.0,756.0,561.0,397.0
2,0500000US34013,793555.0,NaN,209992.0,3292.0,411611.0,2517.0,22146.0,22414.0,22677.0,...,47149.0,46326.0,3775.0,3412.0,17987.0,18203.0,60833.0,2348.0,2279.0,3322.0
3,0500000US34029,591939.0,NaN,46813.0,1622.0,272945.0,1912.0,13955.0,13811.0,13155.0,...,4831.0,8483.0,986.0,1795.0,7443.0,6163.0,8711.0,1581.0,1253.0,1459.0
4,0500000US34031,504041.0,NaN,148834.0,3324.0,251608.0,1859.0,14815.0,14519.0,13334.0,...,38510.0,15509.0,2725.0,1867.0,11181.0,8416.0,20659.0,1613.0,1424.0,2031.0


In [15]:
#Year 1 (Current Year) ACS calculations
#Foreign-born Population - calc & rename
dfY1['PopFB_Y1E'] = dfY1['DP02_0092E'] #check that this variable is total FB and not total pop
dfY1['PopFB_Y1M'] = dfY1['DP02_0092M']
dfY1['PopFB_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['PopFB_Y1E'],x['PopFB_Y1M'])),axis=1)

#Foreign-born % Share of Population
dfY1['PopFBP_Y1E'] = dfY1.apply(lambda x: (calc.get_pct(x['PopFB_Y1E'],x['DP05_0001E'])),axis=1)
dfY1['PopFBP_Y1M'] = dfY1.apply(lambda x: (calc.get_pctmoe(x['PopFB_Y1E'],x['PopFB_Y1M'],\
                                            x['DP05_0001E'],x['DP05_0001M'])),axis=1)
dfY1['PopFBP_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['PopFBP_Y1E'],x['PopFBP_Y1M'])),axis=1)

#Total Labor Force, MOE & CV
dfY1['LFTot_Y1E'] = dfY1['DP03_0002E']
dfY1['LFTot_Y1M'] = dfY1['DP03_0002M']
dfY1['LFTot_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LFTot_Y1E'],x['LFTot_Y1M'])),axis=1)

#Prime-age (25-54) Labor Force, MOE & CV
dfY1['LF2554_Y1E'] = dfY1.loc[:,LF2554E].sum(axis=1)
dfY1['LF2554_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF2554M])),axis=1)
dfY1['LF2554_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF2554_Y1E'],x['LF2554_Y1M'])),axis=1)

#Age 65+ Labor Force, MOE & CV
dfY1['LFO65_Y1E'] = dfY1.loc[:,LFO65E].sum(axis=1)
dfY1['LFO65_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LFO65M])),axis=1)
dfY1['LFO65_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LFO65_Y1E'],x['LFO65_Y1M'])),axis=1)

#Total Housing Units
dfY1['HouTot_Y1E'] = dfY1['DP04_0001E']
dfY1['HouTot_Y1M'] = dfY1['DP04_0001M']
dfY1['HouTot_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['HouTot_Y1E'],x['HouTot_Y1M'])),axis=1)

#Total Housing Units by Tenure - to rename
dfY1['HouO_Y1E'] = dfY1['DP04_0046E']
dfY1['HouO_Y1M'] = dfY1['DP04_0046M']
dfY1['HouO_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['HouO_Y1E'],x['HouO_Y1M'])),axis=1)
dfY1['HouR_Y1E'] = dfY1['DP04_0047E']
dfY1['HouR_Y1M'] = dfY1['DP04_0047M']
dfY1['HouR_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['HouR_Y1E'],x['HouR_Y1M'])),axis=1)
dfY1['HouV_Y1E'] = dfY1['DP04_0003E']
dfY1['HouV_Y1M'] = dfY1['DP04_0003M']
dfY1['HouV_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['HouV_Y1E'],x['HouV_Y1M'])),axis=1)

#Total Housing Units by Building Size
dfY1['Hou1U_Y1E'] = dfY1.loc[:,Hou1UE].sum(axis=1)
dfY1['Hou1U_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Hou1UM])),axis=1)
dfY1['Hou1U_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Hou1U_Y1E'],x['Hou1U_Y1M'])),axis=1)

dfY1['Hou24U_Y1E'] = dfY1.loc[:,Hou24UE].sum(axis=1)
dfY1['Hou24U_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Hou24UM])),axis=1)
dfY1['Hou24U_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Hou24U_Y1E'],x['Hou24U_Y1M'])),axis=1)

dfY1['Hou5U_Y1E'] = dfY1.loc[:,Hou5UE].sum(axis=1)
dfY1['Hou5U_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Hou5UM])),axis=1)
dfY1['Hou5U_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Hou5U_Y1E'],x['Hou5U_Y1M'])),axis=1)

In [16]:
coY1 = dfY1.drop(var_data_Y1[1:],axis=1)
coY1.head()

,GEO_ID,PopFB_Y1E,PopFB_Y1M,PopFB_Y1C,PopFBP_Y1E,PopFBP_Y1M,PopFBP_Y1C,LFTot_Y1E,LFTot_Y1M,LFTot_Y1C,...,HouV_Y1C,Hou1U_Y1E,Hou1U_Y1M,Hou1U_Y1C,Hou24U_Y1E,Hou24U_Y1M,Hou24U_Y1C,Hou5U_Y1E,Hou5U_Y1M,Hou5U_Y1C
0,0500000US34025,83067.0,2180.0,1.595372,0.133251,NaN,NaN,333518.0,2003.0,0.365086,...,6.236710,196813.0,4478.303697,1.383228,17519.0,2361.712303,8.195053,44867.0,3498.167234,4.739665
1,0500000US34037,11236.0,686.0,3.711475,0.078961,NaN,NaN,80059.0,814.0,0.618085,...,10.920367,54408.0,1805.251229,2.017014,2425.0,744.103487,18.653301,4472.0,1021.697607,13.888479
2,0500000US34013,209992.0,3292.0,0.952996,0.264622,NaN,NaN,411611.0,2517.0,0.371732,...,5.647080,128316.0,4129.815613,1.956518,93475.0,5088.454480,3.309211,97023.0,4662.899205,2.921564
3,0500000US34029,46813.0,1622.0,2.106291,0.079084,NaN,NaN,272945.0,1912.0,0.425840,...,3.521335,244717.0,4016.135580,0.997650,13314.0,2047.979736,9.350852,22317.0,2489.628687,6.781611
4,0500000US34031,148834.0,3324.0,1.357666,0.295282,NaN,NaN,251608.0,1859.0,0.449148,...,13.371352,82342.0,3098.487534,2.287507,54019.0,3303.227815,3.717287,40256.0,2958.801447,4.468063


## Labor Force Init Year (Year 0)

In [17]:
co_5 = get_acs(year0,acs5,b=col_5_b,d=col_5_d)
co_5.head()

,B23001_007M,B23001_008E,B23001_008M,B23001_009E,B23001_009M,B23001_010E,B23001_010M,B23001_011E,B23001_011M,B23001_012E,...,state_x,county_x,DP02_0092E,DP02_0092M,DP03_0002E,DP03_0002M,DP05_0001E,DP05_0001M,state_y,county_y
114,489,2563,375,15090,616,10747,665,7152,538,0,...,09,001,182327,3052,475682,2304,905342,-555555555,09,001
116,231,555,161,2756,281,1675,254,1477,245,11,...,09,005,12059,875,107531,846,189916,-555555555,09,005
118,565,3130,361,15241,710,12585,827,8446,624,0,...,09,009,97623,2751,465571,2465,856688,-555555555,09,009
2165,460,1613,265,16018,547,9144,592,5697,515,11,...,34,003,254459,3079,475551,2700,896482,-555555555,34,003
2170,383,2598,369,15835,662,12263,687,7723,625,31,...,34,013,184327,3631,399973,2710,780872,-555555555,34,013


In [18]:
dfY0 = clean_data(co_5,var_data_Y0)

In [19]:
#2010 Population & Labor Force calculations - FINAL TABLE
#Total Labor Force, MOE & CV
dfY0['LFTot_Y0E'] = dfY0['DP03_0002E']
dfY0['LFTot_Y0M'] = dfY0['DP03_0002M']
dfY0['LFTot_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LFTot_Y0E'],x['LFTot_Y0M'])),axis=1)

#Prime-age (25-54) Labor Force, MOE & CV
dfY0['LF2554_Y0E'] = dfY0.loc[:,LF2554E].sum(axis=1)
dfY0['LF2554_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF2554M])),axis=1)
dfY0['LF2554_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF2554_Y0E'],x['LF2554_Y0M'])),axis=1)

#Age 65+ Labor Force, MOE & CV
dfY0['LFO65_Y0E'] = dfY0.loc[:,LFO65E].sum(axis=1)
dfY0['LFO65_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LFO65M])),axis=1)
dfY0['LFO65_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LFO65_Y0E'],x['LFO65_Y0M'])),axis=1)

In [20]:
coY0 = dfY0.drop(var_data_Y0[1:],axis=1)
coY0.head()

,GEO_ID,LFTot_Y0E,LFTot_Y0M,LFTot_Y0C,LF2554_Y0E,LF2554_Y0M,LF2554_Y0C,LFO65_Y0E,LFO65_Y0M,LFO65_Y0C
114,0500000US09001,475682.0,2304.0,0.294442,320032.0,1467.318302,0.278718,25310.0,849.463360,2.040265
116,0500000US09005,107531.0,846.0,0.478267,69780.0,643.345941,0.560464,5955.0,390.621300,3.987569
118,0500000US09009,465571.0,2465.0,0.321859,305076.0,1476.017276,0.294115,21524.0,844.734278,2.385785
2165,0500000US34003,475551.0,2700.0,0.345144,321307.0,1700.218221,0.321676,27134.0,916.883308,2.054160
2170,0500000US34013,399973.0,2710.0,0.411882,278122.0,1768.662206,0.386584,16367.0,755.379375,2.805628


## Change between 2006-2010 5YR (Year 0) and 2014-2018 5YR (Year 1)

In [21]:
#Merge Year 1 and Year 0 into table
coY0Y1 = pd.merge(df_pep,pd.merge(coY0,coY1,how='left',on='GEO_ID'),how='left',on='GEO_ID')

In [22]:
coY0Y1

,GEO_ID,PopTot_Y0E,PopTot_Y1E,PopTot_Y0Y1E,PopP_Y0Y1E,LFTot_Y0E,LFTot_Y0M,LFTot_Y0C,LF2554_Y0E,LF2554_Y0M,...,HouV_Y1C,Hou1U_Y1E,Hou1U_Y1M,Hou1U_Y1C,Hou24U_Y1E,Hou24U_Y1M,Hou24U_Y1C,Hou5U_Y1E,Hou5U_Y1M,Hou5U_Y1C
0,0500000US09001,916829.0,943823.0,26994.0,0.029443,475682.0,2304.0,0.294442,320032.0,1467.318302,...,6.627796,235227.0,4490.525582,1.160497,62217.0,4283.318340,4.185096,75646.0,4091.517445,3.288006
1,0500000US09005,189927.0,181111.0,-8816.0,-0.046418,107531.0,846.0,0.478267,69780.0,643.345941,...,6.701549,68315.0,2074.417991,1.845926,12073.0,1890.105817,9.517108,7663.0,1332.860458,10.573529
2,0500000US09009,862477.0,857620.0,-4857.0,-0.005631,465571.0,2465.0,0.321859,305076.0,1476.017276,...,6.164477,216593.0,4717.644009,1.324082,79679.0,4087.295928,3.118360,69799.0,4362.056625,3.799060
3,0500000US34003,905116.0,936692.0,31576.0,0.034886,475551.0,2700.0,0.345144,321307.0,1700.218221,...,8.556905,208425.0,4556.414380,1.328947,72351.0,4416.514576,3.710814,76981.0,4429.002258,3.497490
4,0500000US34013,783969.0,799767.0,15798.0,0.020151,399973.0,2710.0,0.411882,278122.0,1768.662206,...,5.647080,128316.0,4129.815613,1.956518,93475.0,5088.454480,3.309211,97023.0,4662.899205,2.921564
5,0500000US34017,634266.0,676061.0,41795.0,0.065895,352106.0,2380.0,0.410901,259205.0,1438.927031,...,6.732071,40503.0,2902.028256,4.355606,99210.0,4476.006479,2.742643,143810.0,5494.384406,2.322545
6,0500000US34019,128349.0,124714.0,-3635.0,-0.028321,69179.0,1119.0,0.983309,46005.0,643.370033,...,21.760282,42804.0,1331.181806,1.890545,3749.0,896.862308,14.542679,3762.0,888.450899,14.356505
7,0500000US34021,366513.0,369811.0,3298.0,0.008998,192465.0,1650.0,0.521154,128885.0,1110.987849,...,9.849905,99628.0,3959.325826,2.415872,13353.0,2052.763260,9.345319,31578.0,3314.298870,6.380301
8,0500000US34023,809858.0,829685.0,19827.0,0.024482,426415.0,2239.0,0.319195,296916.0,1384.857754,...,9.893141,188071.0,4730.893890,1.529169,37228.0,3180.941527,5.194217,74788.0,4556.368071,3.703574
9,0500000US34025,630380.0,621354.0,-9026.0,-0.014318,332855.0,1979.0,0.361431,222930.0,1127.957003,...,6.236710,196813.0,4478.303697,1.383228,17519.0,2361.712303,8.195053,44867.0,3498.167234,4.739665


In [23]:
#Calculate change between Year 1 and Year 0, MOE & CVs
#Total Labor Force Change, MOE & CV
coY0Y1['LFTot_Y0Y1E'] = coY0Y1.LFTot_Y1E - coY0Y1.LFTot_Y0E
coY0Y1['LFTot_Y0Y1M'] = coY0Y1.apply(lambda x: (calc.get_moe([x['LFTot_Y0M'],x['LFTot_Y1M']])),axis=1)
coY0Y1['LFTot_Y0Y1C'] = coY0Y1.apply(lambda x: (calc.get_cv(x['LFTot_Y0Y1E'],x['LFTot_Y0Y1M'])),axis=1)

#Prime-age (25-54) Labor Force Change, MOE & CV
coY0Y1['LF2554_Y0Y1E'] = coY0Y1.LF2554_Y1E - coY0Y1.LF2554_Y0E
coY0Y1['LF2554_Y0Y1M'] = coY0Y1.apply(lambda x: (calc.get_moe([x['LF2554_Y0M'],x['LF2554_Y1M']])),axis=1)
coY0Y1['LF2554_Y0Y1C'] = coY0Y1.apply(lambda x: (calc.get_cv(x['LF2554_Y0Y1E'],x['LF2554_Y0Y1M'])),axis=1)

#Age 65+ Labor Force Change, MOE & CV
coY0Y1['LFO65_Y0Y1E'] = coY0Y1.LFO65_Y1E - coY0Y1.LFO65_Y0E
coY0Y1['LFO65_Y0Y1M'] = coY0Y1.apply(lambda x: (calc.get_moe([x['LFO65_Y0M'],x['LFO65_Y1M']])),axis=1)
coY0Y1['LFO65_Y0Y1C'] = coY0Y1.apply(lambda x: (calc.get_cv(x['LFO65_Y0Y1E'],x['LFO65_Y0Y1M'])),axis=1)

#coY0Y1.head()

In [24]:
coY0Y1

,GEO_ID,PopTot_Y0E,PopTot_Y1E,PopTot_Y0Y1E,PopP_Y0Y1E,LFTot_Y0E,LFTot_Y0M,LFTot_Y0C,LF2554_Y0E,LF2554_Y0M,...,Hou5U_Y1C,LFTot_Y0Y1E,LFTot_Y0Y1M,LFTot_Y0Y1C,LF2554_Y0Y1E,LF2554_Y0Y1M,LF2554_Y0Y1C,LFO65_Y0Y1E,LFO65_Y0Y1M,LFO65_Y0Y1C
0,0500000US09001,916829.0,943823.0,26994.0,0.029443,475682.0,2304.0,0.294442,320032.0,1467.318302,...,3.288006,35598.0,3266.844349,5.578751,-6060.0,2092.895363,20.994667,10038.0,1301.004996,7.878905
1,0500000US09005,189927.0,181111.0,-8816.0,-0.046418,107531.0,846.0,0.478267,69780.0,643.345941,...,10.573529,-4718.0,1258.705684,16.218114,-11500.0,888.061372,4.694391,2395.0,607.859359,15.428784
2,0500000US09009,862477.0,857620.0,-4857.0,-0.005631,465571.0,2465.0,0.321859,305076.0,1476.017276,...,3.799060,-5410.0,3575.517445,40.176836,-23349.0,2220.205171,5.780414,8298.0,1361.591348,9.974875
3,0500000US34003,905116.0,936692.0,31576.0,0.034886,475551.0,2700.0,0.345144,321307.0,1700.218221,...,3.497490,23098.0,3690.563101,9.712977,-6701.0,2314.967818,21.000974,10289.0,1499.506919,8.859504
4,0500000US34013,783969.0,799767.0,15798.0,0.020151,399973.0,2710.0,0.411882,278122.0,1768.662206,...,2.921564,11638.0,3698.565803,19.319198,-1372.0,2391.691661,105.970547,6005.0,1135.076209,11.490690
5,0500000US34017,634266.0,676061.0,41795.0,0.065895,352106.0,2380.0,0.410901,259205.0,1438.927031,...,2.322545,23200.0,3360.176186,8.804570,18321.0,2093.947946,6.947856,2061.0,997.056167,29.408693
6,0500000US34019,128349.0,124714.0,-3635.0,-0.028321,69179.0,1119.0,0.983309,46005.0,643.370033,...,14.356505,-23.0,1379.056562,3644.922853,-6532.0,801.872184,7.462650,1697.0,490.603710,17.574504
7,0500000US34021,366513.0,369811.0,3298.0,0.008998,192465.0,1650.0,0.521154,128885.0,1110.987849,...,6.380301,1378.0,2367.636796,104.447960,-7182.0,1501.850525,12.712045,3642.0,839.289581,14.008963
8,0500000US34023,809858.0,829685.0,19827.0,0.024482,426415.0,2239.0,0.319195,296916.0,1384.857754,...,3.703574,3532.0,3037.750813,52.283608,-8176.0,2011.575751,14.956487,7372.0,1129.763692,9.316148
9,0500000US34025,630380.0,621354.0,-9026.0,-0.014318,332855.0,1979.0,0.361431,222930.0,1127.957003,...,4.739665,663.0,2815.750344,258.175315,-21710.0,1623.592621,4.546229,7780.0,1131.578543,8.841770


In [25]:
coY0Y1['stco'] = coY0Y1['GEO_ID'].str[9:]
coY0Y1.head()

,GEO_ID,PopTot_Y0E,PopTot_Y1E,PopTot_Y0Y1E,PopP_Y0Y1E,LFTot_Y0E,LFTot_Y0M,LFTot_Y0C,LF2554_Y0E,LF2554_Y0M,...,LFTot_Y0Y1E,LFTot_Y0Y1M,LFTot_Y0Y1C,LF2554_Y0Y1E,LF2554_Y0Y1M,LF2554_Y0Y1C,LFO65_Y0Y1E,LFO65_Y0Y1M,LFO65_Y0Y1C,stco
0,0500000US09001,916829.0,943823.0,26994.0,0.029443,475682.0,2304.0,0.294442,320032.0,1467.318302,...,35598.0,3266.844349,5.578751,-6060.0,2092.895363,20.994667,10038.0,1301.004996,7.878905,09001
1,0500000US09005,189927.0,181111.0,-8816.0,-0.046418,107531.0,846.0,0.478267,69780.0,643.345941,...,-4718.0,1258.705684,16.218114,-11500.0,888.061372,4.694391,2395.0,607.859359,15.428784,09005
2,0500000US09009,862477.0,857620.0,-4857.0,-0.005631,465571.0,2465.0,0.321859,305076.0,1476.017276,...,-5410.0,3575.517445,40.176836,-23349.0,2220.205171,5.780414,8298.0,1361.591348,9.974875,09009
3,0500000US34003,905116.0,936692.0,31576.0,0.034886,475551.0,2700.0,0.345144,321307.0,1700.218221,...,23098.0,3690.563101,9.712977,-6701.0,2314.967818,21.000974,10289.0,1499.506919,8.859504,34003
4,0500000US34013,783969.0,799767.0,15798.0,0.020151,399973.0,2710.0,0.411882,278122.0,1768.662206,...,11638.0,3698.565803,19.319198,-1372.0,2391.691661,105.970547,6005.0,1135.076209,11.490690,34013


#### Import 31CR Region Geography Tables for Subregion/Region Agg

In [26]:
#Read in geography cross-walk file & clean id column for join
geo_xwalk = pd.read_excel('../data/31CR_CoxSub.xlsx')
geo_xwalk['stco'] = geo_xwalk['stco'].apply(lambda x: '{0:0>5}'.format(x))

#geo_reg.head()

## County

In [27]:
#Merge xwalk file to reduce to region counties only & drop unneeded identifiers
co_cen = geo_xwalk.merge(coY0Y1,on='stco').drop(columns=['st','co','stco_int','reg','subreg','stco_lbl','co_lbl'])

In [28]:
#calculate population density

In [29]:
#Cleanup for MetroExplorer
co_cen = co_cen.drop(columns=['stco']).replace(np.nan,0)
co_cen.set_index('GEO_ID',inplace=True)

In [30]:
for column_name in co_cen.columns:
    co_cen.rename(columns={column_name:column_name.replace('Y0',year0[2:]).replace('Y1',year1[2:])},inplace=True)

In [31]:
co_cen

,PopTot_10E,PopTot_18E,PopTot_1018E,PopP_1018E,LFTot_10E,LFTot_10M,LFTot_10C,LF2554_10E,LF2554_10M,LF2554_10C,...,Hou5U_18C,LFTot_1018E,LFTot_1018M,LFTot_1018C,LF2554_1018E,LF2554_1018M,LF2554_1018C,LFO65_1018E,LFO65_1018M,LFO65_1018C
GEO_ID,,,,,,,,,,,,,,,,,,,,,
0500000US09001,916829.0,943823.0,26994.0,0.029443,475682.0,2304.0,0.294442,320032.0,1467.318302,0.278718,...,3.288006,35598.0,3266.844349,5.578751,-6060.0,2092.895363,20.994667,10038.0,1301.004996,7.878905
0500000US09005,189927.0,181111.0,-8816.0,-0.046418,107531.0,846.0,0.478267,69780.0,643.345941,0.560464,...,10.573529,-4718.0,1258.705684,16.218114,-11500.0,888.061372,4.694391,2395.0,607.859359,15.428784
0500000US09009,862477.0,857620.0,-4857.0,-0.005631,465571.0,2465.0,0.321859,305076.0,1476.017276,0.294115,...,3.799060,-5410.0,3575.517445,40.176836,-23349.0,2220.205171,5.780414,8298.0,1361.591348,9.974875
0500000US34003,905116.0,936692.0,31576.0,0.034886,475551.0,2700.0,0.345144,321307.0,1700.218221,0.321676,...,3.497490,23098.0,3690.563101,9.712977,-6701.0,2314.967818,21.000974,10289.0,1499.506919,8.859504
0500000US34013,783969.0,799767.0,15798.0,0.020151,399973.0,2710.0,0.411882,278122.0,1768.662206,0.386584,...,2.921564,11638.0,3698.565803,19.319198,-1372.0,2391.691661,105.970547,6005.0,1135.076209,11.490690
0500000US34017,634266.0,676061.0,41795.0,0.065895,352106.0,2380.0,0.410901,259205.0,1438.927031,0.337466,...,2.322545,23200.0,3360.176186,8.804570,18321.0,2093.947946,6.947856,2061.0,997.056167,29.408693
0500000US34019,128349.0,124714.0,-3635.0,-0.028321,69179.0,1119.0,0.983309,46005.0,643.370033,0.850139,...,14.356505,-23.0,1379.056562,3644.922853,-6532.0,801.872184,7.462650,1697.0,490.603710,17.574504
0500000US34021,366513.0,369811.0,3298.0,0.008998,192465.0,1650.0,0.521154,128885.0,1110.987849,0.524012,...,6.380301,1378.0,2367.636796,104.447960,-7182.0,1501.850525,12.712045,3642.0,839.289581,14.008963
0500000US34023,809858.0,829685.0,19827.0,0.024482,426415.0,2239.0,0.319195,296916.0,1384.857754,0.283534,...,3.703574,3532.0,3037.750813,52.283608,-8176.0,2011.575751,14.956487,7372.0,1129.763692,9.316148


In [32]:
#co_cen.columns

## Subregion

In [37]:
#Merge geo info with county-level information before calculating change over time
subregion_cen = geo_xwalk.merge(coY0Y1,on='stco')
subregion_cen = subregion_cen.drop(columns=['stco','st','co','stco_int','reg','stco_lbl','co_lbl','GEO_ID'])
subregion_cen.head()

,subreg,PopTot_Y0E,PopTot_Y1E,PopTot_Y0Y1E,PopP_Y0Y1E,LFTot_Y0E,LFTot_Y0M,LFTot_Y0C,LF2554_Y0E,LF2554_Y0M,...,Hou5U_Y1C,LFTot_Y0Y1E,LFTot_Y0Y1M,LFTot_Y0Y1C,LF2554_Y0Y1E,LF2554_Y0Y1M,LF2554_Y0Y1C,LFO65_Y0Y1E,LFO65_Y0Y1M,LFO65_Y0Y1C
0,CT,916829.0,943823.0,26994.0,0.029443,475682.0,2304.0,0.294442,320032.0,1467.318302,...,3.288006,35598.0,3266.844349,5.578751,-6060.0,2092.895363,20.994667,10038.0,1301.004996,7.878905
1,CT,189927.0,181111.0,-8816.0,-0.046418,107531.0,846.0,0.478267,69780.0,643.345941,...,10.573529,-4718.0,1258.705684,16.218114,-11500.0,888.061372,4.694391,2395.0,607.859359,15.428784
2,CT,862477.0,857620.0,-4857.0,-0.005631,465571.0,2465.0,0.321859,305076.0,1476.017276,...,3.799060,-5410.0,3575.517445,40.176836,-23349.0,2220.205171,5.780414,8298.0,1361.591348,9.974875
3,INJ,905116.0,936692.0,31576.0,0.034886,475551.0,2700.0,0.345144,321307.0,1700.218221,...,3.497490,23098.0,3690.563101,9.712977,-6701.0,2314.967818,21.000974,10289.0,1499.506919,8.859504
4,INJ,783969.0,799767.0,15798.0,0.020151,399973.0,2710.0,0.411882,278122.0,1768.662206,...,2.921564,11638.0,3698.565803,19.319198,-1372.0,2391.691661,105.970547,6005.0,1135.076209,11.490690


In [38]:
subregion_cen = geo.calculate_sumgeo(subregion_cen,'subreg')
subregion_cen = subregion_cen.drop(columns = var_agg_todrop)

In [39]:
#Final clean up for MetroExplorer
subregion_cen = subregion_cen.rename(columns={'subreg':'GEO_ID'}).replace(np.nan,0)
subregion_cen.set_index('GEO_ID',inplace=True)
for column_name in subregion_cen.columns:
    subregion_cen.rename(columns={column_name:column_name.replace('Y0',year0[2:]).replace('Y1',year1[2:])},inplace=True)

In [40]:
subregion_cen

,Hou1U_18C,Hou1U_18E,Hou1U_18M,Hou24U_18C,Hou24U_18E,Hou24U_18M,Hou5U_18C,Hou5U_18E,Hou5U_18M,HouO_18C,...,PopFBP_18C,PopFBP_18E,PopFBP_18M,PopFB_18C,PopFB_18E,PopFB_18M,PopP_1018E,PopTot_10E,PopTot_1018E,PopTot_18E
GEO_ID,,,,,,,,,,,,,,,,,,,,,
CT,0.798894,520135.0,6835.509857,2.453786,153969.0,6214.925905,2.432819,153108.0,6127.362402,0.767299,...,0.0,0.418160,0.0,0.884645,324399.0,4720.785634,-0.022607,1969233.0,13321.0,1982554.0
INJ,0.659504,996643.0,10812.416659,1.460145,430532.0,10341.115510,1.293908,532867.0,11341.963807,0.600833,...,0.0,2.362090,0.0,0.348840,1543003.0,8854.403594,0.217607,4986654.0,142320.0,5128974.0
ONJ,0.700673,673686.0,7764.948615,4.503086,54258.0,4019.202906,3.091924,112584.0,5726.265100,0.689073,...,0.0,0.697694,0.0,0.904578,243775.0,3627.446071,-0.073654,1959766.0,4342.0,1964108.0
NYC,1.047444,559901.0,9647.343883,1.026308,778833.0,13148.857061,0.502546,2173992.0,17972.138576,0.712494,...,0.0,1.711358,0.0,0.232356,3127289.0,11953.276957,0.129641,8175133.0,223615.0,8398748.0
MidHud,1.024525,284765.0,4799.268382,4.560418,47388.0,3554.994655,4.373134,51687.0,3718.261960,1.128286,...,0.0,0.393778,0.0,1.401343,95161.0,2193.659500,-0.035922,930341.0,-575.0,929766.0
LI,0.528410,857955.0,7457.642322,4.183527,74609.0,5134.518673,3.001055,111726.0,5515.617554,0.568076,...,0.0,0.377391,0.0,0.609047,532342.0,5333.444009,0.005835,2832882.0,6554.0,2839436.0
LowHud,1.201673,286181.0,5657.091125,3.954141,77430.0,5036.481708,2.286211,155842.0,5860.932946,0.970859,...,0.0,0.593542,0.0,0.828735,327826.0,4469.150590,0.056230,1360510.0,31689.0,1392199.0


## Region

In [41]:
region_cen = geo_xwalk.merge(coY0Y1,on='stco')
region_cen = region_cen.drop(columns=['stco','st','co','stco_int','subreg','stco_lbl','co_lbl','GEO_ID'])

In [42]:
region_cen = geo.calculate_sumgeo(region_cen,'reg')
region_cen = region_cen.drop(columns = var_agg_todrop)

In [43]:
region_cen = region_cen.rename(columns={'reg':'GEO_ID'}).replace(np.nan,0)
region_cen.set_index('GEO_ID',inplace=True)
for column_name in region_cen.columns:
    region_cen.rename(columns={column_name:column_name.replace('Y0',year0[2:]).replace('Y1',year1[2:])},inplace=True)

In [44]:
region_cen

,Hou1U_18C,Hou1U_18E,Hou1U_18M,Hou24U_18C,Hou24U_18E,Hou24U_18M,Hou5U_18C,Hou5U_18E,Hou5U_18M,HouO_18C,...,PopFBP_18C,PopFBP_18E,PopFBP_18M,PopFB_18C,PopFB_18E,PopFB_18M,PopP_1018E,PopTot_10E,PopTot_1018E,PopTot_18E
GEO_ID,,,,,,,,,,,,,,,,,,,,,
31CR,0.3008,4179266.0,20679.70759,0.750919,1617019.0,19974.429278,0.452566,3291806.0,24506.542677,0.283568,...,0.0,6.554015,0.0,0.172794,6193795.0,17605.582893,0.27713,22214519.0,421266.0,22635785.0


# Save as intermediate csvs for later join

In [45]:
#region_county_lf.to_csv('region_county_lf.csv')
co_cen.to_csv('../output/county/co_cen.csv')
subregion_cen.to_csv('../output/subregion/subregion_cen.csv')
region_cen.to_csv('../output/region/region_cen.csv')